In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
dev_image = 'Grazioso Salvare Logo.png'
encoded_dev_image = base64.b64encode(open(dev_image, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.B(html.H3('Roderick Fisher SNHU CS-340 Dashboard')),
   

    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'), style={'display':'inline-block'}),
    
    
    html.Hr(),

    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset'
    )

    ),
    html.Hr(),
             
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        style_header= {
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 0,
        },
        
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
     html.Br(),
     html.Hr(),
     html.Div(className='row',
          style={'display' : 'flex'},
              children=[
         html.Div(
             id='graph-id',
             className='col s12 m6',
             ),
         html.Div(
             id='map-id',
             className='col s12 m6',
            )
         ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(data_frame=dff, 
                            values = values,
                            names = names,
                            color_discrete_sequence=px.colors.sequential.RdBu,
                            width=800,
                            height=500
                        )
                    )
                ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    row_ids[0] = row_ids[len(row_ids) - 1]
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]
app